In [ ]:
!pip install pandas scikit-learn matplotlib


In [ ]:
from google.cloud import bigquery
import os

# 設置服務帳戶憑據
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/final-445615-7590bb2d51fb.json"

# 初始化 BigQuery 客戶端
client = bigquery.Client()

# 執行查詢並保存為 CSV
query = """
SELECT *
FROM `final-445615.ml_data.stackoverflow_data_sample10`
"""
df = client.query(query).to_dataframe()
df.to_csv("stackoverflow_data_10_percent.csv", index=False)
print("Data exported to local file: stackoverflow_data_10_percent.csv")


Data exported to local file: stackoverflow_data_10_percent.csv


In [ ]:
import pandas as pd

# 加載數據
df = pd.read_csv("stackoverflow_data_10_percent.csv")

# 查看數據的前幾行
print(df.head())

# 查看數據的總筆數和欄位
print(f"數據筆數: {len(df)}")
print(f"欄位名稱: {list(df.columns)}")


   answer_score  answer_comment_count  post_type_id  question_view_count  \
0             0                    15             1                   94   
1             0                    11             1                 1571   
2             0                    10             1                   90   
3             0                     9             1                  546   
4             0                    11             1                 1197   

   answerer_reputation  classification_label  
0                    1                     1  
1                 1793                     1  
2                 3330                     0  
3                35843                     1  
4                 2564                     1  
數據筆數: 2134672
欄位名稱: ['answer_score', 'answer_comment_count', 'post_type_id', 'question_view_count', 'answerer_reputation', 'classification_label']


# 數據預處理

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 選擇特徵和目標欄位
features = ["answer_score", "answer_comment_count", "post_type_id", "question_view_count", "answerer_reputation"]
target = "classification_label"

# 分離特徵與目標
X = df[features]
y = df[target]

# 分割數據集為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 標準化數據
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# 初始化分類模型
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# 訓練模型
clf.fit(X_train, y_train)

# 測試模型
y_pred = clf.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.6706805485612564
Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.60      0.62    194265
           1       0.69      0.73      0.71    232670

    accuracy                           0.67    426935
   macro avg       0.67      0.66      0.67    426935
weighted avg       0.67      0.67      0.67    426935



# 用20%的資料，並加上回答者年資特徵 : 回答時間-註冊時間

In [ ]:
from google.cloud import bigquery
import os

# 設置服務帳戶憑據
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/final-445615-7590bb2d51fb.json"

# 初始化 BigQuery 客戶端
client = bigquery.Client()
# 執行查詢並保存為 CSV
query = """
SELECT *
FROM `final-445615.ml_data.stackoverflow_data_sample20`
"""
df = client.query(query).to_dataframe()
df.to_csv("stackoverflow_data_20.csv", index=False)
print("Data exported to local file: stackoverflow_data_20.csv")

Data exported to local file: stackoverflow_data_20.csv


In [ ]:
import pandas as pd

# 加載數據
df = pd.read_csv("stackoverflow_data_20.csv")


# 查看數據的總筆數和欄位
print(f"數據筆數: {len(df)}")
print(f"欄位名稱: {list(df.columns)}")


數據筆數: 4270046
欄位名稱: ['answer_score', 'answer_comment_count', 'post_type_id', 'question_view_count', 'answerer_reputation', 'classification_label', 'user_tenure_seconds']


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 選擇特徵和目標欄位
features = ["answer_score", "answer_comment_count", "post_type_id", "question_view_count", "answerer_reputation","user_tenure_seconds"]
target = "classification_label"

# 分離特徵與目標
X = df[features]
y = df[target]

# 分割數據集為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 標準化數據
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# 初始化分類模型
clf = RandomForestClassifier(n_estimators=50, random_state=42)

# 訓練模型
clf.fit(X_train, y_train)

# 測試模型
y_pred = clf.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.6974812941300453
Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.61      0.65    387508
           1       0.70      0.77      0.74    466502

    accuracy                           0.70    854010
   macro avg       0.70      0.69      0.69    854010
weighted avg       0.70      0.70      0.70    854010



# 用10% accepted_answer_id IS NOT NULL的資料訓練，純數值欄位有vote

In [ ]:
from google.cloud import bigquery
import os

# 設置服務帳戶憑據
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/final-445615-7590bb2d51fb.json"

# 初始化 BigQuery 客戶端
client = bigquery.Client()
# 執行查詢並保存為 CSV
query = """
SELECT *
FROM `final-445615.ml_data.1231_sample10`
"""
full_df = client.query(query).to_dataframe()
full_df.to_csv("stackoverflow_data.csv", index=False)
print("Data exported to local file: stackoverflow_data.csv")

Data exported to local file: stackoverflow_data.csv


In [ ]:
# 查看數據的總筆數和欄位
print(f"數據筆數: {len(full_df)}")
print(f"欄位名稱: {list(full_df.columns)}")


數據筆數: 2133006
欄位名稱: ['answer_score', 'answer_comment_count', 'answerer_reputation', 'answer_favorite_count', 'answerer_up_votes', 'answerer_down_votes', 'classification_label', 'user_tenure_seconds']


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 選擇特徵和目標欄位
features = ["answer_score", "answer_comment_count", "answerer_reputation","user_tenure_seconds","answerer_up_votes","answerer_down_votes"]
target = "classification_label"

# 分離特徵與目標
X = full_df[features]
y = full_df[target]

# 分割數據集為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 標準化數據
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
# 初始化分類模型
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# 訓練模型
clf.fit(X_train, y_train)

# 測試模型
y_pred = clf.predict(X_test)

# 評估模型
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.6356416519378718
Classification Report:
               precision    recall  f1-score   support

         0.0       0.61      0.55      0.58    193442
         1.0       0.66      0.70      0.68    233160

    accuracy                           0.64    426602
   macro avg       0.63      0.63      0.63    426602
weighted avg       0.63      0.64      0.63    426602



# 20%

In [ ]:
from google.cloud import bigquery
import os

# 設置服務帳戶憑據
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/final-445615-7590bb2d51fb.json"

# 初始化 BigQuery 客戶端
client = bigquery.Client()
# 執行查詢並保存為 CSV
query = """
SELECT *
FROM `final-445615.ml_data.1231_sample20`
"""
full_df = client.query(query).to_dataframe()
full_df.to_csv("stackoverflow_data.csv", index=False)
print("Data exported to local file: stackoverflow_data.csv")

Data exported to local file: stackoverflow_data.csv


In [ ]:
# 查看數據的總筆數和欄位
print(f"數據筆數: {len(full_df)}")
print(f"欄位名稱: {list(full_df.columns)}")


數據筆數: 4266360
欄位名稱: ['answer_score', 'answer_comment_count', 'answerer_reputation', 'answer_favorite_count', 'answerer_up_votes', 'answerer_down_votes', 'classification_label', 'user_tenure_seconds']


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 選擇特徵和目標欄位
features = ["answer_score", "answer_comment_count", "answerer_reputation","user_tenure_seconds","answerer_up_votes","answerer_down_votes"]
target = "classification_label"

# 分離特徵與目標
X = full_df[features]
y = full_df[target]

# 分割數據集為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 標準化數據
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np

# 減少參數範圍和樹的數量
clf = RandomForestClassifier(
    n_estimators=50,         # 減少樹的數量以降低計算量
    max_depth=20,            # 限制最大深度避免過度擬合並降低計算量
    min_samples_split=5,     # 增加樣本分裂限制以降低模型複雜度
    random_state=42,
    n_jobs=-1                # 使用所有可用核心加速計算
)

# 訓練模型
clf.fit(X_train, y_train)

# 預測測試集
y_pred = clf.predict(X_test)

# 性能評估
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)  # 處理可能的零除錯誤
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

# 打印評估指標
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.6592024583016904
Precision: 0.6687613855667235
Recall: 0.7436650397841517
F1 Score: 0.7042270711977566

Classification Report:
               precision    recall  f1-score   support

         0.0       0.64      0.56      0.60    387760
         1.0       0.67      0.74      0.70    465512

    accuracy                           0.66    853272
   macro avg       0.66      0.65      0.65    853272
weighted avg       0.66      0.66      0.66    853272



# 特徵工程



In [ ]:
from google.cloud import bigquery
import os

# 設置服務帳戶憑據
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/final-445615-7590bb2d51fb.json"

# 初始化 BigQuery 客戶端
client = bigquery.Client()
# 執行查詢並保存為 CSV
query = """
SELECT *
FROM `final-445615.ml_data.1231_new_10`
"""
full_df = client.query(query).to_dataframe()
full_df.to_csv("stackoverflow_data.csv", index=False)
print("Data exported to local file: stackoverflow_data.csv")

Data exported to local file: stackoverflow_data.csv


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 選擇特徵和目標欄位
target = "classification_label"
features = [col for col in full_df.columns if col != target]# 動態生成 features 列表，排除目標欄位

# 分離特徵與目標
X = full_df[features]
y = full_df[target]


# One-Hot Encoding 非數值特徵
X = pd.get_dummies(X, columns=['score_range'], drop_first=True)

# 分割數據集為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # 標準化數據
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

## 隨機森林

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import pandas as pd


# 初始化隨機森林分類器
clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)

# 訓練模型
clf.fit(X_train, y_train)

# 預測測試集
y_pred = clf.predict(X_test)

# 性能評估
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

# 打印評估指標
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8907730447480727
Precision: 0.5114029025570145
Recall: 0.07643048956827102
F1 Score: 0.1329858927127325

Classification Report:
               precision    recall  f1-score   support

         0.0       0.90      0.99      0.94     78657
         1.0       0.51      0.08      0.13      9682

    accuracy                           0.89     88339
   macro avg       0.70      0.53      0.54     88339
weighted avg       0.85      0.89      0.85     88339



## XGB

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# 選擇特徵和目標欄位
target = "classification_label"
features = [col for col in full_df.columns if col != target]

# 分離特徵與目標
X = full_df[features]
y = full_df[target]

# 確保目標變數為整數類型
y = y.fillna(0).astype('int64')

# 填補缺失值
X = X.fillna(0)

# One-Hot Encoding 非數值特徵
if 'score_range' in X.columns:
    X = pd.get_dummies(X, columns=['score_range'], drop_first=True)

# 分割數據集為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 獲取數值型列
numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

# 特徵縮放
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

print("Data preprocessing completed successfully!")


Data preprocessing completed successfully!


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import pandas as pd



# 初始化 XGBoost 模型
xgb_clf = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=10,  # 平衡正負樣本（需根據類別比例調整）
    random_state=42,
    use_label_encoder=False,
    eval_metric="logloss"
)

# 訓練模型
xgb_clf.fit(X_train, y_train)

# 預測
y_pred = xgb_clf.predict(X_test)

# 性能評估
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

# 打印評估指標
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("\nClassification Report:\n", classification_report(y_test, y_pred))


NameError: name 'X_train' is not defined

# 調整0/1比例

In [ ]:
from google.cloud import bigquery
import os

# 設置服務帳戶憑據
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/final-445615-7590bb2d51fb.json"

# 初始化 BigQuery 客戶端
client = bigquery.Client()
# 執行查詢並保存為 CSV
query = """
SELECT *
FROM `final-445615.ml_data.1231_new_10_adj`
"""
full_df = client.query(query).to_dataframe()
full_df.to_csv("stackoverflow_data_adj.csv", index=False)
print("Data exported to local file: stackoverflow_data_adj.csv")

Data exported to local file: stackoverflow_data_adj.csv


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 選擇特徵和目標欄位
target = "classification_label"
features = [col for col in full_df.columns if col != target]# 動態生成 features 列表，排除目標欄位

# 分離特徵與目標
X = full_df[features]
y = full_df[target]


# One-Hot Encoding 非數值特徵
X = pd.get_dummies(X, columns=['score_range'], drop_first=True)

# 分割數據集為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import pandas as pd


# 初始化隨機森林分類器
clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=20,
    min_samples_split=10,
    random_state=42,
    n_jobs=-1
)

# 訓練模型
clf.fit(X_train, y_train)

# 預測測試集
y_pred = clf.predict(X_test)

# 性能評估
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

# 打印評估指標
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7318550879940651
Precision: 0.7043725960843287
Recall: 0.7237757042672055
F1 Score: 0.7139423429233046

Classification Report:
               precision    recall  f1-score   support

         0.0       0.76      0.74      0.75     78274
         1.0       0.70      0.72      0.71     67304

    accuracy                           0.73    145578
   macro avg       0.73      0.73      0.73    145578
weighted avg       0.73      0.73      0.73    145578



XGB

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import pandas as pd

# 初始化 XGBoost 分類器
xgb_clf = XGBClassifier(
    n_estimators=100,       # 樹的數量
    max_depth=20,           # 最大樹深度
    learning_rate=0.1,      # 學習率
    subsample=0.8,          # 每棵樹的樣本子集比例
    colsample_bytree=0.8,   # 每棵樹的特徵子集比例
    random_state=42,        # 保持結果一致
    use_label_encoder=False,  # 禁用舊版標籤編碼
    eval_metric="logloss"   # 評估指標
)

# 訓練模型
xgb_clf.fit(X_train, y_train)

# 預測測試集
y_pred = xgb_clf.predict(X_test)

# 性能評估
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

# 打印評估指標
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("\nClassification Report:\n", classification_report(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [10:30:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.7183159543337593
Precision: 0.6897960362024914
Recall: 0.7100172352311899
F1 Score: 0.6997605816328772

Classification Report:
               precision    recall  f1-score   support

         0.0       0.74      0.73      0.73     78274
         1.0       0.69      0.71      0.70     67304

    accuracy                           0.72    145578
   macro avg       0.72      0.72      0.72    145578
weighted avg       0.72      0.72      0.72    145578



# test

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# 初始化隨機森林分類器，增加 class_weight
clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=20,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1,
    class_weight={0: 1.2, 1: 1}  # 加重類別 0 的懲罰
)

# 訓練模型
clf.fit(X_train, y_train)

# 預測測試集
y_pred = clf.predict(X_test)

# 性能評估
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

# 打印評估指標
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.7286884007198889
Precision: 0.7183510011778563
Recall: 0.6796178533222393
F1 Score: 0.6984478427839578

Classification Report:
               precision    recall  f1-score   support

         0.0       0.74      0.77      0.75     78274
         1.0       0.72      0.68      0.70     67304

    accuracy                           0.73    145578
   macro avg       0.73      0.73      0.73    145578
weighted avg       0.73      0.73      0.73    145578



In [ ]:
import pandas as pd
from IPython.display import display, HTML

# 讀取 JSON 檔案
data = pd.read_json("/content/bquxjob_443d9b92_1941cf22ec2.json")

# 確認使用的特徵列
features = [
    "answer_comment_count",
    "answerer_reputation",
    "answerer_net_votes",
    "answerer_total_ans_count",
    "user_tenure",
    "score_range"
]

# 獲取特徵數據
X = data[features]

# One-Hot Encoding 非數值特徵
X = pd.get_dummies(X, columns=['score_range'], drop_first=True)

# 確保特徵順序與訓練一致
expected_columns = clf.feature_names_in_  # 確認模型訓練時的特徵名稱
for col in expected_columns:
    if col not in X.columns:
        X[col] = 0  # 如果缺失特徵，補充為 0
X = X[expected_columns]  # 按訓練時的順序排列

# 使用 clf 模型進行預測
predicted_probs = clf.predict_proba(X)[:, 1]  # 預測目標為 1 的概率

# 添加燈號
def get_light(prob):
    if prob >= 0.7:
        return "綠燈"
    elif prob >= 0.3:
        return "黃燈"
    else:
        return "無燈"

data["Predicted Probability"] = predicted_probs
data["Light"] = data["Predicted Probability"].apply(get_light)

# 添加小球樣式
def add_ball(light):
    if light == "綠燈":
        color = "#28a745"  # 綠色
    elif light == "黃燈":
        color = "#ffc107"  # 黃色
    elif light == "無燈":
        color = "#E0E0E0"  # 無色
    else:
        color = "red"  # 默認紅色
    return f'<div style="width: 16px; height: 16px; background-color: {color}; border-radius: 50%; margin: auto;"></div>'

data["Light Ball"] = data["Light"].apply(add_ball)

# 選擇要顯示的列
output_columns = [
    "classification_label",
    "Predicted Probability",
    "Light Ball"
]

output_data = data[output_columns]

# 將 DataFrame 轉換為 HTML
html_table = output_data.to_html(escape=False, index=False)

# 顯示帶有圓形小球的表格
display(HTML(html_table))


classification_label,Predicted Probability,Light Ball
0,0.174535,
0,0.059761,
1,0.372554,
0,0.000000,
0,0.625798,
0,0.707985,
0,0.469712,
1,0.696179,
0,0.166518,
0,0.035294,


In [ ]:
import pandas as pd
from IPython.display import display, HTML

# 讀取 JSON 檔案
data = pd.read_json("/content/bquxjob_52fa1ef7_1941cd3a625.json")

# 確認使用的特徵列
features = [
    "answer_comment_count",
    "answerer_reputation",
    "answerer_net_votes",
    "answerer_total_ans_count",
    "user_tenure",
    "score_range"
]

# 獲取特徵數據
X = data[features]

# One-Hot Encoding 非數值特徵
X = pd.get_dummies(X, columns=['score_range'], drop_first=True)

# 確保特徵順序與訓練一致
expected_columns = clf.feature_names_in_  # 確認模型訓練時的特徵名稱
for col in expected_columns:
    if col not in X.columns:
        X[col] = 0  # 如果缺失特徵，補充為 0
X = X[expected_columns]  # 按訓練時的順序排列

# 使用 clf 模型進行預測
y_pred = clf.predict(X)  # 預測類別

# 添加燈號

def get_light(pred):
    if pred == 1:
        return "綠燈"
    else:
        return "無燈"

data["Predicted Class"] = y_pred
data["Light"] = data["Predicted Class"].apply(get_light)

# 添加小球樣式
def add_ball(light):
    if light == "綠燈":
        color = "#28a745"  # 綠色
    elif light == "無燈":
        color = "#E0E0E0"  # 無色
    else:
        color = "red"  # 默認紅色
    return f'<div style="width: 16px; height: 16px; background-color: {color}; border-radius: 50%; margin: auto;"></div>'

data["Light Ball"] = data["Light"].apply(add_ball)

# 選擇要顯示的列
output_columns = [
    "classification_label",
    "Predicted Class",
    "Light Ball"
]

output_data = data[output_columns]

# 將 DataFrame 轉換為 HTML
html_table = output_data.to_html(escape=False, index=False)

# 顯示帶有圓形小球的表格
display(HTML(html_table))


classification_label,Predicted Class,Light Ball
0,0.0,
0,0.0,
0,0.0,
0,0.0,
1,0.0,
0,0.0,
1,0.0,
0,0.0,
0,0.0,
0,0.0,


In [ ]:
import pandas as pd
from IPython.display import display, HTML

# 讀取 JSON 檔案
data = pd.read_json("/content/bquxjob_510393a6_1941ceeb87c.json")

# 確認使用的特徵列
features = [
    "answer_comment_count",
    "answerer_reputation",
    "answerer_net_votes",
    "answerer_total_ans_count",
    "user_tenure",
    "score_range"
]

# 獲取特徵數據
X = data[features]

# One-Hot Encoding 非數值特徵
X = pd.get_dummies(X, columns=['score_range'], drop_first=True)

# 確保特徵順序與訓練一致
expected_columns = clf.feature_names_in_  # 確認模型訓練時的特徵名稱
for col in expected_columns:
    if col not in X.columns:
        X[col] = 0  # 如果缺失特徵，補充為 0
X = X[expected_columns]  # 按訓練時的順序排列

# 使用 clf 模型進行預測
predicted_probs = clf.predict_proba(X)[:, 1]  # 預測目標為 1 的概率

# 計算總體平均概率
mean_prob = predicted_probs.mean()

# 添加燈號
def get_light(prob):
    if prob >= mean_prob + 0.2:  # 高於平均值 0.25 的為綠燈
        return "綠燈"
    elif prob >= mean_prob :  # 在平均值 ±0.5 之間的為黃燈
        return "黃燈"
    else:
        return "無燈"

data["Predicted Probability"] = predicted_probs
data["Light"] = data["Predicted Probability"].apply(get_light)

# 添加小球樣式
def add_ball(light):
    if light == "綠燈":
        color = "#28a745"  # 綠色
    elif light == "黃燈":
        color = "#ffc107"  # 黃色
    elif light == "無燈":
        color = "#E0E0E0"  # 無色
    else:
        color = "red"  # 默認紅色
    return f'<div style="width: 16px; height: 16px; background-color: {color}; border-radius: 50%; margin: auto;"></div>'

data["Light Ball"] = data["Light"].apply(add_ball)

# 選擇要顯示的列
output_columns = [
    "classification_label",
    "Light Ball"
]

output_data = data[output_columns]

# 將 DataFrame 轉換為 HTML
html_table = output_data.to_html(escape=False, index=False)

# 顯示帶有圓形小球的表格
display(HTML(html_table))


classification_label,Light Ball
0,
1,
1,
0,
1,
1,
0,
0,
0,
1,


NLP TEST


In [ ]:
from google.cloud import bigquery
import os

# 設置服務帳戶憑據
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/final-445615-7590bb2d51fb.json"

# 初始化 BigQuery 客戶端
client = bigquery.Client()
# 執行查詢並保存為 CSV
query = """
SELECT *
FROM `final-445615.ml_data.1231_new_001`
"""
full_df = client.query(query).to_dataframe()
full_df.to_csv("stackoverflow_data_001.csv", index=False)
print("Data exported to local file: stackoverflow_data_001.csv")

Data exported to local file: stackoverflow_data_001.csv


In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from bs4 import BeautifulSoup

# 讀取從 BigQuery 匯出的 CSV 檔案
data_path = "stackoverflow_data_001.csv"  # 確保與導出的檔案名稱一致
df = pd.read_csv(data_path)

# 清理文本的函數
def clean_text(text):
    """清理 HTML 並提取文本"""
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text() if text else ""

# 特徵提取函數
def extract_features(row):
    """針對每一行數據提取特徵"""
    question_title = row['question_title']
    question_body = row['question_body']
    answer_body = row['answer_body']

    # 清理文本
    q_title_clean = clean_text(question_title)
    q_body_clean = clean_text(question_body)
    a_body_clean = clean_text(answer_body)

    # 基本特徵
    features = {
        'question_length': len(q_title_clean.split()) + len(q_body_clean.split()),
        'answer_length': len(a_body_clean.split()),
        'num_code_blocks': len(re.findall(r'<code>.*?</code>', question_body)) + len(re.findall(r'<code>.*?</code>', answer_body)),
        'contains_example_code': 1 if '<code>' in answer_body else 0,
        'contains_links': 1 if re.search(r'http[s]?://', answer_body) else 0,
    }

    # 語義相似度 (TF-IDF)
    texts = [q_title_clean, q_body_clean, a_body_clean]
    tfidf = TfidfVectorizer().fit_transform(texts)
    features['similarity_title_answer'] = cosine_similarity(tfidf[0], tfidf[2])[0, 0]
    features['similarity_body_answer'] = cosine_similarity(tfidf[1], tfidf[2])[0, 0]

    return features

# 對所有數據提取特徵
feature_list = []
for index, row in df.iterrows():
    feature_list.append(extract_features(row))

# 轉為 DataFrame
features_df = pd.DataFrame(feature_list)

# 合併原始數據和特徵
final_df = pd.concat([df, features_df], axis=1)

# 保存結果為 CSV
final_df.to_csv("stackoverflow_features.csv", index=False)
print("Features extracted and saved to stackoverflow_features.csv")


<ipython-input-28-320d776a28b3>:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
